In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from dotenv import load_dotenv

In [5]:

load_dotenv()

os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

1. Loading PDFs and chunking with LangChain

In [22]:
# #add PDFs

# #simple method - split by pages
# loader = PyPDFLoader("Bhagavad-gita-Swami-BG-Narasingha.pdf")
# pages = loader.load_and_split()
# print(pages[0])

# #skip to step 2 if you're using this method
# chunks = pages

page_content='with commentar y by' metadata={'source': 'Bhagavad-gita-Swami-BG-Narasingha.pdf', 'page': 2}


In [ ]:
# step 1: convert PDF to text (using PyPDF2)
import PyPDF2

def extract_text_from_pdf(pdf_path):
    text = ""
    pdf_file_obj = open(pdf_path, "rb")
    pdf_reader = PyPDF2.PdfFileReader(pdf_file_obj)
    for page_num in range(pdf_reader.numPages):
        page_obj = pdf_reader.getPage(page_num)
        text += page_obj.extractText()
    pdf_file_obj.close()
    return text

doc = extract_text_from_pdf("Bhagavad-gita-Swami-BG-Narasingha.pdf")

# step 2: Save to .txt and reopen
with open("Bhagavad-gita-Swami-BG-Narasingha.txt", 'w') as f:
    f.write(doc)

with open("Bhagavad-gita-Swami-BG-Narasingha.txt", 'r') as f:
    text = f.read()

# step 3: Create function to count tokens (no changes)
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

# step 4: Split text into chunks (no changes)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])
